In [1]:
import tensorflow as tf 
import numpy as np
import keras
import keras.backend as K 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, GlobalMaxPooling1D, MaxPooling2D
from keras.optimizers import Adam
#from keras.utils.training_utils import multi_gpu_model

Using TensorFlow backend.


In [2]:
import numpy
import csv
import h5py
import gc 
import argparse
import os 
import psutil

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpu_options = tf.GPUOptions(allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


In [4]:
n_hidden_units = 50
n_convpool_layers = 1
n_convlayers = 2
n_reglayers = 2
max_poolsize = (2,2)
train_set_ratio = 0.7
valid_set_ratio = 0.15
set_split_ratio34 = 0.5
drop_rate = 0.0
n_filters = 130 #130 before 
reg_factor = 0.0
kernel_size = (2,2)
input_strides = 1
kernel_init = 'uniform'
cost_function = 'mean_squared_error'
batch_size = 64#128
feed_batch_size = 256
input_sample_shape = [128,128,5]
#target_sample_shape = [50]
n_epochs = 35
n_output = [0,50]
n_set1 = 10000
n_set2 = None
n_set100 = 10000 
n_post_test = 5000
xnorm_axis = (0,1,2,3)
early_stop_delta = 0.01 # 0.01 change or above is considered improvement
early_stop_patience = 10 


In [7]:
def create_model():
    model=Sequential()

    model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
                     input_shape=(128,128,5),kernel_initializer= kernel_init))
    model.add(Activation('relu'))

    #model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
    #                 kernel_initializer= kernel_init))
    #model.add(Activation('relu'))

    #model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
    #                 kernel_initializer= kernel_init))
    #model.add(Activation('relu'))


    model.add(MaxPooling2D(pool_size=max_poolsize))

    #model.add(BatchNormalization())

    model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
                     kernel_initializer= kernel_init))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=max_poolsize))

    #model.add(BatchNormalization())

    model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
                     kernel_initializer= kernel_init))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=max_poolsize))
    
#     model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
#                      kernel_initializer= kernel_init))
#     model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=max_poolsize))

    model.add(Flatten())
    model.add(Dense(n_hidden_units))
    model.add(Activation('relu'))
    model.add(Dense(50, activation='linear'))

    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=early_stop_delta, patience=early_stop_patience, verbose=2, mode='auto')

    model.summary()
    return model

In [8]:
model = create_model()
model.compile(loss=cost_function, optimizer='adam', metrics=["MAE"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 127, 127, 130)     2730      
_________________________________________________________________
activation_5 (Activation)    (None, 127, 127, 130)     0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 130)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 62, 62, 130)       67730     
_________________________________________________________________
activation_6 (Activation)    (None, 62, 62, 130)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 130)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 130)       67730     
__________

In [9]:
ds = h5py.File("../data/data2D%d.h5","r",driver='family', memb_size=2500*10**6)["random"]

In [10]:
X = ds["x0"][0:15000]

Y = ds["y0"][0:15000]

In [11]:
print(X.shape)
print(Y.shape)

(15000, 128, 128, 5)
(15000, 50)


In [12]:
# x_mean = np.mean(X,axis=(0,1,2),dtype=X.dtype)
# x_max = np.amax(X,axis=(0,1,2))
# # Modulus norm
# X,xmean,xxrange = data_normalize_v2_modulus(X,norm_type = 'max',data_mean= x_mean,data_range= x_max[0],norm_axis=(0,1,2), mat_axis=0)
# # Density norm
# X,xmean,xxrange = data_normalize_v2_modulus(X,norm_type = 'max',data_mean= x_mean,data_range= x_max[1],norm_axis=(0,1,2), mat_axis=1)

x_mean = np.mean(X,axis=(0,1,2),dtype=X.dtype)
x_max = np.amax(X,axis=(0,1,2))
x_min = np.amin(X,axis=(0,1,2))
x_data_normed = (X-x_mean)/x_max

In [13]:
y_mean = np.mean(Y)
y_max = np.amax(Y,axis=())
y_min = np.amin(Y,axis = ())
y_data_normed = (Y-y_mean)/y_max

In [14]:
print(x_data_normed.shape)
print(y_data_normed.shape)

(15000, 128, 128, 5)
(15000, 50)


In [ ]:
X_train,Y_train = x_data_normed[:12000],y_data_normed[:12000]
#model.load_weights("model_12000.h5")
model.fit(X_train,Y_train, epochs=25, batch_size=64, verbose=1)

W0804 19:12:38.946366 139674320226048 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0804 19:12:39.082068 139674320226048 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/25


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


 3008/12000 [======>.......................] - ETA: 38s - loss: 1.4985 - mean_absolute_error: 0.4749

In [102]:
model.save("model_12000_all_normed_c4_f130.h5")

In [69]:
X_test,Y_test = x_data_normed[12000:],y_data_normed[12000:]
inputMatrix = np.load("../data/inputMatrix.npz")
Solution = np.load("../data/Solution.npz")

In [70]:
print("------inputMatrix.npz------")
for key in list(inputMatrix.keys()):
    print(key+":",end = "")
    print(inputMatrix[key].shape)
print("\n------Solution.npz------")
for key in list(Solution.keys()):
    print(key+":",end = "")
    print(Solution[key].shape)

------inputMatrix.npz------
inputMatrix:(22, 128, 128, 5)

------Solution.npz------
Qp:(22, 2)
frequency1:(22, 882)
Mp:()


In [71]:
inputMatrix = inputMatrix["inputMatrix"]
Solution = np.real(Solution[list(Solution.keys())[1]][:,0:50])
print(inputMatrix.shape)
print(Solution.shape)

(22, 128, 128, 5)
(22, 50)


In [72]:
in_mean = np.mean(inputMatrix,axis=(0,1,2))
print(in_mean.shape)

(5,)


In [73]:
in_max = np.amax(inputMatrix,axis=(0,1,2))
print(in_max.shape)

(5,)


In [74]:
out_mean = np.mean(Solution,axis = (0))
print(out_mean.shape)

(50,)


In [75]:
out_max = np.amax(Solution,axis = (0))
print(out_max.shape)

(50,)


In [106]:
# normed_input,_,_=data_normalize_v2_modulus(inputMatrix,norm_type="max",data_mean=in_mean,data_range=in_max,norm_axis=(0,1,2),mat_axis=0)
# normed_input,_,_=data_normalize_v2_modulus(inputMatrix,norm_type="max",data_mean=in_mean,data_range=in_max,norm_axis=(0,1,2),mat_axis=1)
# # normed_input,_,_=data_normalize_v2_modulus(inputMatrix,norm_type="max",data_mean=in_mean,data_range=in_max,norm_axis=(0,1,2),mat_axis=2)
# # normed_input,_,_=data_normalize_v2_modulus(inputMatrix,norm_type="max",data_mean=in_mean,data_range=in_max,norm_axis=(0,1,2),mat_axis=3)
# # normed_input,_,_=data_normalize_v2_modulus(inputMatrix,norm_type="max",data_mean=in_mean,data_range=in_max,norm_axis=(0,1,2),mat_axis=4)
# print(normed_input.shape)
x_test_normed = (inputMatrix-x_mean)/x_max
y_test_normed = (Solution-y_mean)/y_max
print(x_test_normed.shape)
print(y_test_normed.shape)

ValueError: operands could not be broadcast together with shapes (22,50) (15000,50) 

In [43]:
# normed_output,_,_ = data_normalize_y(Solution,norm_type="max",data_mean=out_mean,data_range=out_max)
# print(normed_output.shape)

(22, 50)


In [77]:
loss,mae=model.evaluate(x_test_normed,y_test_normed)
print(loss)
print(mae)

22/22 [==============================] - 0s 3ms/step
0.14336954057216644
0.10070051997900009


1.8406311811786182e+20

16565.7126171875